### Cross Validation K-Folds *(After Pre-Process Data)*

In [ ]:
%%script true
# The accuracy At Each Eval fold Represents The Accuracy of The Best Method With the Relative Training Fold
# There is no guarantee that the same method will be the one with best accuracy multiple times
# The Objective Is To Calculate the mean Accuracy Presented by These differentes Methods Selected By Differents Parts of the DB

k = 7

results = {}
train_method_quality = {}
kfolds_eval_method_quality = {}

folds_condos_imgs_idxs = [
    [3, 7],
    [0, 1],
    [4, 12],
    [5, 9],
    [6, 11],
    [2, 13],
    [8, 10]
]


for eval_fold_i in range(k):
    
    # Defining Condos Idx For Training & Eval
    eval_condo_idxs = folds_condos_imgs_idxs[eval_fold_i]
    
    train_condo_idxs = []
    for fold_i in range(len(folds_condos_imgs_idxs)):
        if (fold_i == eval_fold_i): continue
        
        train_condo_idxs = train_condo_idxs + folds_condos_imgs_idxs[fold_i]
        
        
    # Param Tuning For Each Method & Space
    for color_space in color_space_oriented_df:
        
        # Initing Quality Tracking Objects
        if not color_space in train_method_quality:
            train_method_quality[color_space] = {}
            
        if not color_space in kfolds_eval_method_quality:
            kfolds_eval_method_quality[color_space] = {}
            
            
        # Extracting Quality Measurements Of Each Method
        for method_name in color_space_oriented_df[color_space].keys():
            method_analysis_data = copy.deepcopy(color_space_oriented_df[color_space][method_name])
            
            # Select Training Condos
            method_analysis_data_df = pd.DataFrame(method_analysis_data).iloc[train_condo_idxs]
                
            # Getting Quality Features Table
            method_analysis_class_metrics_df = reduce_df_dim(method_analysis_data_df, "classification_evaluation_metrics")
            
            # Saving Quality Features
            train_method_quality[color_space][method_name] = {}
            train_method_quality[color_space][method_name][sort_metric] = mean_w_precision(method_analysis_class_metrics_df[sort_metric])


        # Selecting Best Method w/ Training DS
        color_spaces_methods_quality_df = pd.DataFrame(train_method_quality[color_space]).transpose()
        
        color_spaces_methods_quality_df_no_uncertainty = color_spaces_methods_quality_df.copy(deep=True)
        
        # FIXME
        # Ordering By Quality ---
        # color_spaces_methods_quality_df_no_uncertainty[sort_metric] = color_spaces_methods_quality_df_no_uncertainty[sort_metric].apply(lambda x: float(x))
        # Ordering By Complmentary Uncertainty ---
        color_spaces_methods_quality_df_no_uncertainty[sort_metric] = color_spaces_methods_quality_df_no_uncertainty[sort_metric].apply(lambda x: 100 - x.uncertainty)
        
        best_method = color_spaces_methods_quality_df_no_uncertainty.nlargest(1, sort_metric).iloc[0]
        best_method_name = best_method.name
        
        # Quality of Best Method w/ Eval DS
        best_method_analysis_data = color_space_oriented_df[color_space].loc[[best_method_name]].values[0]
        best_method_analysis_eval_folds_df = pd.DataFrame(best_method_analysis_data).loc[eval_condo_idxs]
        best_method_w_eval_fold_class_metrics = reduce_df_dim(best_method_analysis_eval_folds_df, "classification_evaluation_metrics")
        
        kfolds_eval_method_quality[color_space][eval_fold_i] = {}
        for metric in metrics_to_track:
            kfolds_eval_method_quality[color_space][eval_fold_i][metric] = mean_w_precision(best_method_w_eval_fold_class_metrics[metric])

In [ ]:
%%script true
# Inject Mean Fold Metrics  ---

for color_space in kfolds_eval_method_quality:
    best_methods_at_folds_df = pd.DataFrame(kfolds_eval_method_quality[color_space]).transpose()
    
    # Converted To NpArray, So Mean Is Calculated maintaning NumberWithUncertainty
    for metric in metrics_to_track:
        kfolds_eval_method_quality[color_space][f"mean_{metric}"] = np.array(best_methods_at_folds_df[metric].to_list()).mean()

In [ ]:
%%script true
# [Best Eval Methods] Mean CV-K-folds Quality considering All Color Spaces ---

mean_kfolds_quality_all_color_spaces = []

for color_space in kfolds_eval_method_quality.keys():
    mean_kfolds_quality_all_color_spaces.append(kfolds_eval_method_quality[color_space][f"mean_{sort_metric}"])
    
print(f"[Best Eval Methods] Mean CV-K-folds Quality considering All Color Spaces:")
print(f"  {np.array(mean_kfolds_quality_all_color_spaces).mean()}")